In [2]:
!pip install geopandas

  Using cached geopandas-0.11.1-py3-none-any.whl (1.0 MB)
  Using cached pyproj-3.3.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
     |████████████████████████████████| 2.1 MB 10.8 MB/s            
  Using cached Fiona-1.8.21-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.6 MB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)


In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, LineString, Point
from shapely.geometry import shape
import concurrent.futures
import time
import pandas as pd
import fiona
import json


In [2]:
dataset = pd.read_csv('Building_Permits_v7.csv', index_col='Unnamed: 0', low_memory=False)#'Unnamed')

In [3]:
dataset.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Est_Cost_Infl,Rev_Cost_Infl,Est_Cost_Infl_log10,Est_Cost_Infl_loge,lat_lon,address.1,geometry,point,polygon,area_m2
0,9111378,2,new construction wood frame,1991-06-28,3566,037,3686,17th,St,erect a four story three unit residential bldg,...,4.973752e+05,696325.256975,5.696684,13.117100,-4623.268159,"3686 17th St, San Francisco, CA 94114",NaN,POINT (-122.42798037718552 37.76316610254898),"POLYGON ((6004384.44156398 2106033.407080037, ...",147.915331
1,8707430,2,new construction wood frame,1987-05-29,1733,046,1254,18th,Av,NaN,...,9.398079e+05,NaN,5.973039,13.753431,-4625.233438,"1254 18th Av, San Francisco, CA 94122","{""type"": ""Polygon"", ""coordinates"": [[[-122.476...",POINT (-122.47578567375196 37.76447248121622),"POLYGON ((5990594.490913103 2106871.242348627,...",253.922359
2,7711072S,2,new construction wood frame,1977-10-19,0121,032,1059,Union,St,to erect 12 unit apartment,...,7.476268e+06,NaN,6.873685,15.827244,-4627.254130,"1059 Union St, San Francisco, CA 94133",NaN,POINT (-122.41653445746154 37.79925767816894),NaN,NaN
3,7912377,2,new construction wood frame,1979-12-01,1361,064,3900,California,St,NaN,...,2.948572e+06,NaN,6.469612,14.896831,-4627.238361,"3900 California St, San Francisco, CA 94118","{""type"": ""Polygon"", ""coordinates"": [[[-122.459...",POINT (-122.45952470081986 37.7858592269433),POLYGON ((5995398.966155565 2114500.1454920806...,504.669823
4,8010466,2,new construction wood frame,1980-11-25,4159,074,1084,De Haro,St,NaN,...,6.922231e+05,NaN,5.840246,13.447664,-4621.384747,"1084 De Haro St, San Francisco, CA 94107",NaN,POINT (-122.40107686660524 37.7560791545319),POLYGON ((6012157.402672291 2103366.9430118585...,167.842843


In [4]:
# Refactoring Zipcode
dataset['Zipcode'] = dataset['Zipcode'].replace(np.nan, 0.0).astype(str)
dataset['Zipcode'] = dataset['Zipcode'].apply(lambda x: x[:-2])
dataset['Zipcode'] = dataset['Zipcode'].replace('0','')

In [5]:
#Manipulations on Number of Proposed Stories column
#there are some mistaken values in number of proposed stories. we will use description column to fix some of them
#let's take rows where there less than 1 and more than 15 stories
m_ps1 = dataset['Number of Proposed Stories'] > 15 
m_ps2 = dataset['Number of Proposed Stories'] < 1
m_ps3 = dataset['Number of Proposed Stories'].isna()
#let's find description rows where we have 'story' word
m_ps4 = dataset['Description'].fillna('empty').str.contains('story')
#taking rows (m_ps1 or m_ps1) and m_ps3
m_ps = ( m_ps1 | m_ps2 | m_ps3 ) & m_ps4

def text_split(x):
  #x will be sth similar to 'erect a two story 88 unit residential structure'
  #we do text partition with 'story' 
  #it returns tuple ('erect a two ', 'story', ' 88 unit residential structure')
  #then, we take the first value of tuple 
  #and then apply string manipulations to obtain floor number in text
  return x.partition('story')[0].replace('-',' ').split(' ')[-2]

def text2int (x):
  #converting text to number for the possible cases
  x = x.lower()
  if 'one' in x:
    y = 1
  elif 'two' in x:
    y= 2
  elif 'three' in x:
    y=3
  elif 'four' in x:
    y = 4
  elif 'five' in x:
    y = 5
  elif 'six' in x:
    y = 6
  elif 'seven' in x:
    y = 7
  elif 'eight' in x:
    y = 8
  elif 'nine' in x:
    y = 9
  elif  'ten' in x:
    y = 10
  elif  'eleven' in x:
    y = 11
  else:
    try : 
      y = int(x)
    except :
      y = np.nan
  return y

col_ = 'Number of Proposed Stories'
#a new columns for the manipulation
dataset[col_+ '_'] = dataset[col_]

#adding story numbers on the masked rows
dataset.loc[m_ps,col_+ '_']=   dataset.loc[m_ps,'Description'].apply(lambda x:text_split(x) )
dataset.loc[m_ps,col_+ '_']=   dataset.loc[m_ps,col_+ '_'].apply(lambda x : text2int(x))
dataset.loc[:,col_+ '_']= dataset.loc[:,col_+ '_'].astype(float)

In [6]:
#adding a column with story number categories
def cat_stories (st): 
    if st < 3 :
      y = '0-2 stories'
    elif st< 5 :
      y = '3-4 stories'
    elif st < 8 :
      y = '5-7 stories'
    elif st < 10 :
      y = '8-9 stories'
    else:
      y = 'More than 10 stories'
    return y

col_ ='Number of Proposed Stories'
dataset[col_+'_cat'] = dataset[col_].apply(lambda x: cat_stories(x)).astype(str)

In [7]:
#following masks have been applied after having data analyses performed.
# see the notebok on exploratory data analysis for more details
m_out0 = dataset['Est_Cost_Infl_log10'] <= dataset['Est_Cost_Infl_log10'].quantile(0.99)#8.0
m_out1 = dataset['Est_Cost_Infl_log10'] >= dataset['Est_Cost_Infl_log10'].quantile(0.01)#3.5
m_out2 = dataset['Proposed Units'] <= dataset['Proposed Units'].quantile(0.99)#<= 200
m_out3 = dataset['Proposed Units'] >= dataset['Proposed Units'].quantile(0.01)#> 0
m_out4 = dataset['Number of Proposed Stories_'] <= 15
m_out = m_out0 & m_out1 & m_out2 & m_out3 & m_out4
#removing outliers
dataset=dataset.loc[m_out,:]

In [8]:
dataset.shape

(7360, 49)

In [9]:
def re_category (ds,counts, repl_ ):
    #It replaces the categories that are not sufficiently presented in the dataseries
    #It also fills NaN values with the defined category value
    n_count = ds.value_counts()
    m_ng = ds.isin (n_count.index[n_count.values < counts])
    ds[m_ng] = repl_
    ds.fillna(repl_,inplace=True)
    return ds.astype('str')

#for categories of zipcode with less than 20 data, we use category 'Other'
col_ = 'Zipcode'
dataset[col_+'_']=re_category (dataset[col_] , 20, 'Other' )

#for categories of proposed construction type with less than 20 data, we use category 99
col_ = 'Proposed Construction Type'
dataset[col_+'_']=re_category (dataset[col_] , 20, 'Other' )


#Adding columns with boxcox transformation
#from scipy import stats
#from scipy.stats import norm, skew
from scipy.special import boxcox1p

dataset['Number of Proposed Stories_cat_f']=pd.factorize(dataset['Number of Proposed Stories_cat'])[0]
dataset['Proposed Use_f']=pd.factorize(dataset['Proposed Use'])[0]
dataset['Proposed Construction Type_f']=pd.factorize(dataset['Proposed Construction Type_'])[0]

skewed_features = ['Number of Proposed Stories', 'Number of Proposed Stories_cat_f',
         'Proposed Construction Type_f',  'Proposed Units', 'Proposed Use_f',
         'Duration_construction_days']

lam = 0.10 #lan value obtained after trial and error. If 0 is used, boxcox1p becomes same with np.log1p 
for feat in skewed_features:
    dataset[feat+'_bct'] = boxcox1p(dataset[feat].fillna(dataset[feat].mean()), lam)

/tmp/ipykernel_404/3047112504.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds[m_ng] = repl_


In [10]:
dataset[['Block','Lot','address.1','Zipcode','Est_Cost_Infl','Proposed Units','Number of Proposed Stories_','area_m2','polygon']]#.plot.hist()

,Block,Lot,address.1,Zipcode,Est_Cost_Infl,Proposed Units,Number of Proposed Stories_,area_m2,polygon
0,3566,037,"3686 17th St, San Francisco, CA 94114",94114,4.973752e+05,3.0,4.0,147.915331,"POLYGON ((6004384.44156398 2106033.407080037, ..."
1,1733,046,"1254 18th Av, San Francisco, CA 94122",94122,9.398079e+05,4.0,3.0,253.922359,"POLYGON ((5990594.490913103 2106871.242348627,..."
2,0121,032,"1059 Union St, San Francisco, CA 94133",94133,7.476268e+06,12.0,4.0,NaN,NaN
3,1361,064,"3900 California St, San Francisco, CA 94118",94118,2.948572e+06,8.0,3.0,504.669823,POLYGON ((5995398.966155565 2114500.1454920806...
4,4159,074,"1084 De Haro St, San Francisco, CA 94107",94107,6.922231e+05,2.0,3.0,167.842843,POLYGON ((6012157.402672291 2103366.9430118585...
...,...,...,...,...,...,...,...,...,...
7556,4104,065,"647 Pennsylvania Av, San Francisco, CA 94107",94107,4.571580e+05,2.0,4.0,114.329641,POLYGON ((6016233.245128858 2096161.4738537618...
7557,3553,033,"45 Adair St, San Francisco, CA 94103",94103,6.917408e+05,3.0,3.0,113.618340,"POLYGON ((6016236.920117994 2096112.593564261,..."
7558,6423,015,"1750 Geneva Av, San Francisco, CA 94134",94134,2.506771e+06,16.0,3.0,116.814116,POLYGON ((6016261.816353401 2096114.7397428534...
7559,6423,009,"1600 Birchwood Ct, San Francisco, CA",Other,3.577949e+06,12.0,3.0,110.794146,"POLYGON ((6016287.595950073 2096102.66899015, ..."


In [11]:
dataset['total_area_m2'] = dataset['area_m2'] * dataset['Number of Proposed Stories_']
dataset['cost_per_m2'] = dataset['Est_Cost_Infl'] /dataset['total_area_m2'] 

In [12]:
dataset.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Proposed Use_f,Proposed Construction Type_f,Number of Proposed Stories_bct,Number of Proposed Stories_cat_f_bct,Proposed Construction Type_f_bct,Proposed Units_bct,Proposed Use_f_bct,Duration_construction_days_bct,total_area_m2,cost_per_m2
0,9111378,2,new construction wood frame,1991-06-28,3566,037,3686,17th,St,erect a four story three unit residential bldg,...,0,0,1.746189,0.0,0.000000,1.486984,0.000000,7.659654,591.661324,840.641704
1,8707430,2,new construction wood frame,1987-05-29,1733,046,1254,18th,Av,NaN,...,0,0,1.486984,0.0,0.000000,1.746189,0.000000,7.451394,761.767077,1233.720846
2,7711072S,2,new construction wood frame,1977-10-19,0121,032,1059,Union,St,to erect 12 unit apartment,...,0,1,1.746189,0.0,0.717735,2.923922,0.000000,9.579565,NaN,NaN
3,7912377,2,new construction wood frame,1979-12-01,1361,064,3900,California,St,NaN,...,0,0,1.486984,0.0,0.000000,2.457309,0.000000,8.214724,1514.009469,1947.525221
4,8010466,2,new construction wood frame,1980-11-25,4159,074,1084,De Haro,St,NaN,...,1,0,1.486984,0.0,0.000000,1.161232,0.717735,7.880294,503.528529,1374.744586


In [16]:
dataset.shape

(7360, 62)

In [17]:
dataset.to_csv('Building_Permits_v8bis.csv',index=False)

In [18]:
dataset = dataset.dropna(subset = ['area_m2'])

In [19]:
dataset.shape

(6575, 62)

In [20]:
dataset.to_csv('Building_Permits_v8.csv',index=False)